In [1]:
''' This script is developed to study the evolution of the NPDFS 
    OF MOLECULAR CLOUDS for various initial conditons.
    
    
    Date Created : 20May 2019
    Author  : Sayantan
    
    Model : TPB16V2 
    Using the new TP_module
    This was adopted from the new objected oriented module developed by Sayantan
    
    Output: 
    
    Movie showing the evolution of the PDF
    
    
'''

# import TP_module as mf
import glob
import numpy as np
import time as t
import os.path
import matplotlib.pyplot as plt
from TP_class_module import *
import multiprocessing as mp
from pylab import*

New TP_class_module is imported


In [2]:
Model = "TP_B25_v30"

current_directory = os.getcwd() 
print("the current directory in", current_directory)

save_path1 = './time_evol_plots_'+ Model
save_path = './fit_params_'+ Model
# param_folder= current_directory + '/fit_params_'+ Model
image_folder= current_directory + '/time_evol_plots_'+ Model

# os.path.isdir(path)

## Making the directory for the putput files
try:  
#     os.makedirs(param_folder)
    os.makedirs(image_folder)
except OSError:  
    print ("Creation of the directory %s failed/ not needed as it alread exit")
else:  
    print ("Successfully created the directory %s")

the current directory in /Users/sayantan/Documents/Work/Projects/TRANSITION_PDF_17Dec18/TP_OOPS_25April_19/TP_V2_21_May19
Creation of the directory %s failed/ not needed as it alread exit


In [3]:
list_of_files = glob.glob('../evolution_data/'+ Model +'/*.dat')
list = np.sort(list_of_files)
bin = [50,100,150,200,250,295] 
b = bin[1]

In [4]:
def do_plot(number):
    fileName = list[number]
    fig = plt.figure(number)
    
## for compiling in series for the for-loop and coment the fucntion ###
# for fileName in list[123:150]:    
    print(fileName)
    
### To check the intial data as a plot and to get the xdata and the ydata ###
    MODEL = PDF_analysis(Model, fileName, binsize=150, upperlimit=None)    
    xdata1, ydata1, ydata_not_normalized, norm_const = MODEL.fittingdata()
#     fig = MODEL.plothist()
#     plt.show()    
    
    #     ## to determine the time
    time_step = int(fileName.replace('../evolution_data/'+ Model +'/colden_ev', '').replace('.dat', ''))
    time_in_my = str((np.float(time_step)-10)*2.5*10**(-2))
#     print(time_step)

    ### Model Selection 
    MODEL_FIT_LG = Fitting_the_PDF(Model, fileName, binsize=b,function_to_fit="Lognormal", bounds=((0, 0), (1, 1)))
    MODEL_FIT_PW = Fitting_the_PDF(Model, fileName, binsize=b,function_to_fit="Piecewise", bounds=((0, 0, -7, .5), (1, 1, -2, .7)))
     
    popt_lg, pcov_lg = MODEL_FIT_LG.model_fitting_parameters()
    

    popt, pcov =  MODEL_FIT_PW.model_fitting_parameters()
#     print(r'$\mu$=%5.2f, $\sigma$=%5.2f, $\alpha$=%5.2f, $\eta_{\rm {TP}}$=%5.2f' % tuple(popt))

    ## To get the intial chi quare values for both lognormal and piecewise function it    
    chisq_lg=MODEL_FIT_LG.get_chisquare_value(Best_fit_values=popt_lg)
    chisq_lgpw=MODEL_FIT_PW.get_chisquare_value(Best_fit_values=popt)
#     print("chisq lognormal+powerlaw (LC method)=", chisq_lgpw)
#     print("chisq lognormal(LC method)=",chisq_lg )
    
   
    
    if (chisq_lgpw <= 0.8*chisq_lg or chisq_lg >20 or chisq_lgpw > 20) and popt[2]>-5 and popt[3]< np.max(xdata1):
        
        fig = MODEL_FIT_PW.get_fitted_plot(time_in_my=time_in_my, Method='PYMC',guess_value_for_pymc=popt)
#         plt.show()
    else:        
        fig = MODEL_FIT_LG.get_fitted_plot(time_in_my=time_in_my)
        
    outputfilename_figfile = os.path.join(save_path1,'ev_'+str(time_step)+".jpg")
    plt.savefig(outputfilename_figfile,format='jpg',dpi=300)  
    plt.close()
    

In [5]:
number_of_timestep = len(list)
# start_point = 
# end_point =
print(number_of_timestep)

# number_of_orbit = 10
start = time.time()
print("Plotting now...")
if __name__ == '__main__':
    pool = mp.Pool()
    pool.map(do_plot, range(0,number_of_timestep,1))
print("End of simulation")    
end = t.time()
T = end-start
print("Total time for parallel run %.2f"%T, 'sec')

187
Plotting now...
../evolution_data/TP_B25_v30/colden_ev022.dat
../evolution_data/TP_B25_v30/colden_ev046.dat
../evolution_data/TP_B25_v30/colden_ev034.dat
../evolution_data/TP_B25_v30/colden_ev010.dat


/anaconda3/lib/python3.6/site-packages/scipy/optimize/minpack.py:794: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


../evolution_data/TP_B25_v30/colden_ev035.dat
../evolution_data/TP_B25_v30/colden_ev023.dat
../evolution_data/TP_B25_v30/colden_ev047.dat
../evolution_data/TP_B25_v30/colden_ev011.dat
../evolution_data/TP_B25_v30/colden_ev036.dat
../evolution_data/TP_B25_v30/colden_ev024.dat
../evolution_data/TP_B25_v30/colden_ev048.dat
../evolution_data/TP_B25_v30/colden_ev012.dat
../evolution_data/TP_B25_v30/colden_ev025.dat
../evolution_data/TP_B25_v30/colden_ev037.dat
../evolution_data/TP_B25_v30/colden_ev049.dat
../evolution_data/TP_B25_v30/colden_ev013.dat
../evolution_data/TP_B25_v30/colden_ev026.dat
../evolution_data/TP_B25_v30/colden_ev038.dat
../evolution_data/TP_B25_v30/colden_ev050.dat
../evolution_data/TP_B25_v30/colden_ev014.dat
../evolution_data/TP_B25_v30/colden_ev027.dat
../evolution_data/TP_B25_v30/colden_ev039.dat
../evolution_data/TP_B25_v30/colden_ev051.dat
../evolution_data/TP_B25_v30/colden_ev015.dat
../evolution_data/TP_B25_v30/colden_ev028.dat
../evolution_data/TP_B25_v30/colde

/Users/sayantan/Documents/Work/Projects/TRANSITION_PDF_17Dec18/TP_OOPS_25April_19/TP_V2_21_May19/TP_class_module.py:334: RuntimeWarning: invalid value encountered in log10
  p1 = np.log10(np.log(10) * 1 / (np.sqrt(2 * np.pi) * sigma))


 [--                5%                  ] 1062 of 20000 complete in 0.5 sec../evolution_data/TP_B25_v30/colden_ev076.dat
../evolution_data/TP_B25_v30/colden_ev100.dat
../evolution_data/TP_B25_v30/colden_ev088.dat
 [-------          19%                  ] 3998 of 20000 complete in 2.5 sec../evolution_data/TP_B25_v30/colden_ev077.dat
 [--------         23%                  ] 4642 of 20000 complete in 3.0 sec../evolution_data/TP_B25_v30/colden_ev101.dat
../evolution_data/TP_B25_v30/colden_ev089.dat
 [----------       26%                  ] 5311 of 20000 complete in 3.5 sec

...

/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:224: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  return reshape(newshape, order=order)


 [-----------------58%--                ] 11634 of 20000 complete in 7.5 seccBest fit  using PYMC: mu= 0.57, sigma= 0.34, alpha=-1.81, TP= 0.81
Err in best fit vals using PYMC: mu= 0.01, sigma= 0.00, alpha= 0.33, TP= 0.01
 [-----------------61%---               ] 12376 of 20000 complete in 8.0 secc

/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:224: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  return reshape(newshape, order=order)


../evolution_data/TP_B25_v30/colden_ev066.dat
 [-----------------67%-----             ] 13578 of 20000 complete in 9.0 secBest fit  using PYMC: mu= 0.58, sigma= 0.35, alpha=-4.68, TP= 0.51
Err in best fit vals using PYMC: mu= 0.01, sigma= 0.01, alpha= 0.31, TP= 0.05
 [-----------------71%-------           ] 14211 of 20000 complete in 9.5 sec../evolution_data/TP_B25_v30/colden_ev143.dat
 [--                5%                  ] 1108 of 20000 complete in 0.5 sececBest fit  using PYMC: mu= 0.54, sigma= 0.33, alpha=-3.55, TP= 0.40
Err in best fit vals using PYMC: mu= 0.02, sigma= 0.01, alpha= 0.19, TP= 0.04
 [---               9%                  ] 1947 of 20000 complete in 1.0 secec../evolution_data/TP_B25_v30/colden_ev155.dat
 [--------------   37%                  ] 7521 of 20000 complete in 5.0 sececBest fit  using PYMC: mu= 0.61, sigma= 0.38, alpha=-4.42, TP= 0.44
Err in best fit vals using PYMC: mu= 0.02, sigma= 0.01, alpha= 0.38, TP= 0.08
 [-----------------52%                  ] 10

 [-----------------56%-                 ] 11266 of 20000 complete in 7.5 seccBest fit  using PYMC: mu= 0.58, sigma= 0.36, alpha=-4.61, TP= 0.51
Err in best fit vals using PYMC: mu= 0.01, sigma= 0.01, alpha= 0.27, TP= 0.06
 [-----------------79%----------        ] 15804 of 20000 complete in 11.5 sec../evolution_data/TP_B25_v30/colden_ev192.dat
 [-----------------98%----------------- ] 19685 of 20000 complete in 13.5 secBest fit  using PYMC: mu= 0.53, sigma= 0.32, alpha=-3.50, TP= 0.39
Err in best fit vals using PYMC: mu= 0.02, sigma= 0.01, alpha= 0.21, TP= 0.04
 [-----------------46%                  ] 9313 of 20000 complete in 6.0 sececBest fit  using PYMC: mu= 0.57, sigma= 0.31, alpha=-3.98, TP= 0.42
Err in best fit vals using PYMC: mu= 0.01, sigma= 0.01, alpha= 0.14, TP= 0.03
 [-----------------100%-----------------] 20000 of 20000 complete in 13.2 sec../evolution_data/TP_B25_v30/colden_ev177.dat
 [-----------------68%-----             ] 13608 of 20000 complete in 9.0 secBest fit  us

In [6]:
import cv2
print("Making the Video")

image_folder = save_path1 
video_name = 'video_'+ Model +'.mov'
if os.path.isfile(video_name):
    print("Removing the old video and making the new one")
    os.remove(video_name)

image = [img for img in os.listdir(image_folder) if img.endswith(".jpg")]
## sorting the images in ascending order of time
images= sorted(image, key=lambda x: int(x.replace("ev_", '').replace('.jpg', '')))
# images =sorted(image)
# print(images)
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

video = cv2.VideoWriter(video_name, -1, 13, (width,height))

for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

cv2.destroyAllWindows()
video.release()

print("The video is done")

Making the Video
The video is done


In [7]:
## This is delete all the images to save space
fileList = os.listdir(save_path1)

for fileName in fileList:
#     print(fileName)
    os.remove(save_path1+"/"+fileName)